### Temporal Convolutional Networks Overview

![TCNs](https://cdn-images-1.medium.com/max/1000/1*1cK-UEWHGaZLM-4ITCeqdQ.png)

# Global Parameters

In [1]:
import tensorflow as tf
import numpy as np
from tabulate import tabulate
print(tf.__version__)

1.10.0


In [2]:
#tf.enable_eager_execution()

## Input

In [3]:
suffix = "all"
train_data = np.load("../../data/protein/classification/train_features_"+suffix+".npy")
train_label = np.load("../../data/protein/classification/train_labels_"+suffix+".npy")
val_data = np.load("../../data/protein/classification/val_features_"+suffix+".npy")
val_label = np.load("../../data/protein/classification/val_labels_"+suffix+".npy")

In [4]:
train_data = np.asarray([ np.asarray(element) for element in train_data[:,0]])

In [5]:
val_data = np.asarray([ np.asarray(element) for element in val_data[:,0]])

In [6]:
train_data.shape, val_data.shape

((185137, 600), (46285, 600))

In [7]:
def get_one_label_dict(labels):
    one_label_dict = {}
    i = 0
    for label in labels:
        if label[0] not in one_label_dict:
            one_label_dict[label[0]] = {}
        if label[1] not in one_label_dict[label[0]]:
            one_label_dict[label[0]][label[1]] = {}
        if label[2] not in one_label_dict[label[0]][label[1]]:
            one_label_dict[label[0]][label[1]][label[2]] = {}
        if label[3] in one_label_dict[label[0]][label[1]][label[2]]:
            continue
        else:
            one_label_dict[label[0]][label[1]][label[2]][label[3]] = i
            i = i + 1
    return one_label_dict

In [8]:
def convert_to_one_label(labels, one_label_dict):
    one_labels = []
    for label in labels:
        one_labels.append(one_label_dict[label[0]][label[1]][label[2]][label[3]])
    return one_labels

In [9]:
train_label_one = convert_to_one_label(train_label, get_one_label_dict(train_label))

In [10]:
val_label_one = convert_to_one_label(val_label, get_one_label_dict(train_label))

In [11]:
len(set(train_label_one))

1616

## Building TCNs

###  Causal Convolution

###  Spatial Dropout

Reference: https://stats.stackexchange.com/questions/282282/how-is-spatial-dropout-in-2d-implemented

Actually, simply setting noise_shape in tf.layers.Dropout will do the trick.

In [12]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.random_normal((32, 4, 10)) # (batch_size, channel, length)
    dropout = tf.layers.Dropout(0.5, noise_shape=[x.shape[0], x.shape[1], tf.constant(1)])
    output = dropout(x, training=True)
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output)
    print(res.shape)   
    print(res[0, :, :])
    print(res[1, :, :])

(32, 4, 10)
[[-0.82197386 -3.0046816  -3.724099   -2.8766513  -0.22663254  0.88886863
   1.6210606  -2.3477132   2.278684   -0.6836003 ]
 [-1.937318    0.7735584   2.840088    0.21559832 -0.7209419  -0.32201436
   1.0481603   2.226582    1.1027129  -2.3055673 ]
 [-0.70179737 -0.8198413  -0.43068424 -0.19779772 -0.9129509  -2.2073545
  -1.6410354   1.2270569  -2.4816089   2.0207832 ]
 [-1.310588   -0.9603893  -1.6769583   0.01309513  2.2220554  -0.6717528
   5.1088195   1.8669513   0.55742496  1.5011914 ]]
[[ 0.         -0.          0.          0.          0.          0.
   0.         -0.          0.          0.        ]
 [-0.          0.         -0.          0.         -0.          0.
   0.          0.          0.         -0.        ]
 [-0.          0.          0.         -0.          0.         -0.
  -0.          0.         -0.         -0.        ]
 [-2.6522791  -0.18217683  0.6235265   0.21999747 -0.22957887  1.8100656
  -1.908828   -3.8984725   0.14673786  1.2256571 ]]


### Temporal blocks

Note: `tf.contrib.layers.layer_norm` only supports `channels_last`.

In [13]:
# Redefining CausalConv1D to simplify its return values
class CausalConv1D(tf.layers.Conv1D):
    def __init__(self, filters,
               kernel_size,
               strides=1,
               dilation_rate=1,
               activation=None,
               use_bias=True,
               kernel_initializer=tf.contrib.layers.xavier_initializer(),
               bias_initializer=tf.zeros_initializer(),
               kernel_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               bias_constraint=None,
               trainable=True,
               name=None,
               **kwargs):
        super(CausalConv1D, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding='valid',
            data_format='channels_last',
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            trainable=trainable,
            name=name, **kwargs
        )
       
    def call(self, inputs):
        pad_1 = int(self.kernel_size[0]/2)
        pad_2 = self.kernel_size[0]-1-pad_1
        width_padding = [ pad_1* self.dilation_rate[0], pad_2 * self.dilation_rate[0]]
        print(width_padding)
        inputs = tf.pad(inputs, tf.constant([(0, 0,), width_padding, (0, 0)]), "REFLECT")
        return super(CausalConv1D, self).call(inputs)

In [14]:
class CausalConv2D(tf.layers.Conv2D):
    def __init__(self,
                 filters,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 data_format='channels_last',
                 dilation_rate=(1, 1),
                 activation=None,
                 use_bias=True,
                 kernel_initializer=tf.contrib.layers.xavier_initializer(),
                 bias_initializer=tf.zeros_initializer(),
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 trainable=True,
                 name=None,
                 reuse=None,
                 **kwargs):
        super(CausalConv2D, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=(1,strides),
            padding='valid',
            data_format='channels_last',
            dilation_rate=(1, dilation_rate),
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            trainable=trainable,
            name=name, 
            **kwargs
        )
       
    def call(self, inputs):
        pad_1 = int(self.kernel_size[1]/2)
        pad_2 = self.kernel_size[1]-1-pad_1
        heigh_padding = [ pad_1* self.dilation_rate[1], pad_2 * self.dilation_rate[1]]
        pad_1 = int(self.kernel_size[0]/2)
        pad_2 = self.kernel_size[0]-1-pad_1
        width_padding = [ pad_1* self.dilation_rate[0], pad_2 * self.dilation_rate[0]]
        inputs = tf.pad(inputs, tf.constant([(0, 0,), width_padding, heigh_padding, (0,0)]), "REFLECT")
        return super(CausalConv2D, self).call(inputs)

In [15]:
class TemporalBlock(tf.layers.Layer):
    def __init__(self, n_outputs, kernel_size, strides, dilation_rate, dropout=0.2, trainable=True, name=None, dtype=None, 
                 activity_regularizer=None, **kwargs):
        super(TemporalBlock, self).__init__(trainable=trainable, dtype=dtype, activity_regularizer=activity_regularizer,
                                            name=name, **kwargs)        
        self.dropout = dropout
        self.n_outputs = n_outputs
        self.conv1 = CausalConv2D(n_outputs, kernel_size, strides=strides, dilation_rate=1, 
                                  activation=tf.nn.relu, name="conv1")
        self.conv2 = CausalConv2D(n_outputs, kernel_size, strides=1, dilation_rate=dilation_rate, 
                                  activation=tf.nn.relu, name="conv2")
        
        self.conv3 = CausalConv2D(n_outputs, kernel_size, strides=strides, dilation_rate=1, 
                          activation=tf.nn.relu, name="conv3")

    
    def build(self, input_shape):
        channel_dim = 2
        self.dropout1 = tf.layers.Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
        self.dropout2 = tf.layers.Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
    
    def call(self, inputs, training=True):
        x = self.conv1(inputs)
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout1(x, training=training)
        x = self.conv2(x)
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout2(x, training=training)
        inputs = self.conv3(inputs)
        return tf.nn.leaky_relu(x + inputs)

In [16]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.random_normal((1, 3, 8, 1)) # (batch_size, length, channel)
    tblock = TemporalBlock(8, (3,3), 1, 4) #n_outputs, kernel_size, strides, dilation_rate
    output = tblock(x, training=tf.constant(True))
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output)
    print("Input with shape of {}".format(x.shape))
    print(tabulate(tf.squeeze(x).eval()))
    print("Output with shape of {}".format(res.shape))
    print(tabulate(res[0, :, :, 0]))
    print(tabulate(res[0, :, :, 1]))
    print(tabulate(res[0, :, :, 2]))
    print(tabulate(res[0, :, :, 3]))
    print(tabulate(res[0, :, :, 4]))
    print(tabulate(res[0, :, :, 5]))
    print(tabulate(res[0, :, :, 6]))
    print(tabulate(res[0, :, :, 7]))

Input with shape of (1, 3, 8, 1)
---------  ---------  ----------  ---------  ---------  ----------  ----------  ---------
-1.0154    -0.586145  -0.0847688   1.08107    0.664716   1.14846    -0.0510511   1.07549
 1.00501    0.250471  -0.135105    0.69252    0.429332  -0.0694308  -0.336875   -0.890913
-0.372627  -1.25377    0.726707   -0.951095  -0.285603  -0.170857    0.386026    0.746121
---------  ---------  ----------  ---------  ---------  ----------  ----------  ---------
Output with shape of (1, 3, 8, 8)
---------  ----------  ----------  ---------  ----------  ---------  ----------  ---------
-0.118704   4.65737    -0.101032    1.43941   -0.138552   -0.147065  -0.0242772   3.97687
 2.51609   -0.0844534  -0.0154095  -0.147065  -0.147065    3.66903   -0.147065   -0.115054
-0.147065   5.88011    -0.132755    1.22316   -0.0890022  -0.147065  -0.102725    3.90816
---------  ----------  ----------  ---------  ----------  ---------  ----------  ---------
---------  ---------  ---------

### Temporal convolutional networks

In [34]:
class TemporalConvNet(tf.layers.Layer):
    def __init__(self, num_channels, strides, kernel_size=2, dropout=0.2, trainable=True, name=None, dtype=None, 
                 activity_regularizer=None, **kwargs):
        super(TemporalConvNet, self).__init__(trainable=trainable, dtype=dtype, activity_regularizer=activity_regularizer,
                                              name=name, **kwargs)
        self.layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 1 ** i
            out_channels = num_channels[i]
            self.layers.append(TemporalBlock(out_channels, kernel_size, strides=strides[i], dilation_rate=dilation_size,
                                             dropout=dropout, name="tblock_{}".format(i)))
    
    def call(self, inputs, training=True):
        outputs = inputs
        for layer in self.layers:
            outputs = layer(outputs, training=training)
        return outputs

In [35]:
tf.reset_default_graph()
g = tf.Graph()
with g.as_default():
    Xinput = tf.placeholder(tf.float32, shape=[None, 10, 10, 1])
    tcn = TemporalConvNet([8, 8, 8, 8], [1, 1, 1, 1], (3,3), 0.25)
    output = tcn(Xinput, training=tf.constant(True))
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output, {Xinput: np.random.randn(1, 10, 10, 1)})
    print(res.shape)   
    print(tabulate(res[0, :, 0]))
    print(tabulate(res[0, :, 1]))

(1, 10, 10, 8)
---------  ----------  ---------  ----------  --------  --------  ---------  --------
 2.56614    1.0259      1.53186   -0.133287   2.35915   0         -0.133287  0
 0.766785   2.59044     1.33161   -0.133287   0         0          0.665247  0
-0.133287   0.860084   -0.122152  -0.133287   1.56387   1.5653    -0.133287  0.311007
 0.871284   0.542529    0.559552  -0.123903   1.14042   1.67408   -0.133287  0
-0.133287   2.74805    -0.133287  -0.133287   1.18399   0          1.52117   2.67627
-0.133287   3.64058    -0.133287   0.359349   0.848606  0.969381  -0.133287  0
 1.63504   -0.133287    0.681861  -0.133287   0         0         -0.133287  0
-0.133287  -0.0871876  -0.133287   0.218772   0.870577  0          0.500016  0
-0.133287  -0.0732622  -0.133287  -0.133287   0         0         -0.133287  0
-0.133287   0.0516056  -0.133287  -0.0654511  0.410016  0         -0.133287  0
---------  ----------  ---------  ----------  --------  --------  ---------  --------
----------

# Model

In [36]:
# Training Parameters
learning_rate = 0.001
batch_size = 8
batches_per_epoch = int(train_data.shape[0]/batch_size)+1
num_epochs = 15
print("Number of epochs: {} with batches per epoch: {}".format(num_epochs, batches_per_epoch))

# Network Parameters
sequence_length=train_data.shape[1]
num_classes = len(set(train_label_one))
num_of_acids = 21
embedding_size = 3
dropout = 0.5
kernel_size = (3,3)
levels = 6
hidden_dim_f = 2
hidden_features = [hidden_dim_f, hidden_dim_f*2, hidden_dim_f*4, hidden_dim_f*8, hidden_dim_f*16 ,hidden_dim_f*32] # hidden layer num of features
strides = [1 ,2, 2, 2, 2, 2]

Number of epochs: 15 with batches per epoch: 23143


In [37]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(10)
    
    with tf.variable_scope('input'):
        sequences = tf.placeholder(tf.int32, [None, sequence_length], name='sequences')
        labels = tf.placeholder(tf.int32, (None,))
        is_training = tf.placeholder(tf.bool, name='is_train')

        dataset = (tf.data.Dataset.from_tensor_slices((sequences, labels))
                   .shuffle(buffer_size=10000, reshuffle_each_iteration=True)
                   .apply(tf.contrib.data.batch_and_drop_remainder(batch_size)))
    
        iterator = dataset.make_initializable_iterator()
        
    
    with tf.variable_scope('embedding'):
        acid_embeddings = tf.get_variable("acid_embeddings", [num_of_acids, embedding_size])

        batch_sequences, batch_labels = iterator.get_next()

        embedded_sequences = tf.nn.embedding_lookup(acid_embeddings, batch_sequences)
        embedded_sequences = tf.expand_dims(tf.transpose(embedded_sequences, perm=[0,2,1]), axis=3)
        print(embedded_sequences.shape)
    # Define weights
    with tf.variable_scope('tcn'):
        tcn_output = TemporalConvNet(hidden_features, strides, kernel_size, 
                                                 dropout)(embedded_sequences, training=is_training)
        print("tcn_output", tcn_output.shape)
        flatten_tcn_output = tf.layers.flatten(tcn_output)
        print("flatten_tcn_output", flatten_tcn_output.shape)
        logits = tf.layers.dense(flatten_tcn_output,num_classes, activation=None, kernel_initializer=tf.orthogonal_initializer())
            

    # Define loss and optimizer
    with tf.name_scope("loss_op"):
        loss_op = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels=batch_labels, logits=logits))
        tf.summary.scalar("loss_op", loss_op)
    
    with tf.name_scope("optimizer"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op)

    with tf.name_scope("accuracy"):
        prediction = tf.nn.softmax(logits)
        correct_pred = tf.equal(tf.argmax(prediction, 1, output_type=tf.int32), tf.squeeze(batch_labels))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar("accuracy", accuracy)
    
    summ = tf.summary.merge_all()
    
     # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    print("All parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.global_variables()]))
    print("Trainable parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.trainable_variables()]))
    [ print("{}{}".format(x.name, x.shape)) for x in tf.trainable_variables() if "LayerNorm" not in x.name]

(8, 3, 600, 1)
tcn_output (8, 3, 19, 64)
flatten_tcn_output (8, 3648)
All parameters: 17988029.0
Trainable parameters: 5996009
embedding/acid_embeddings:0(21, 3)
tcn/temporal_conv_net/tblock_0/conv1/kernel:0(3, 3, 1, 2)
tcn/temporal_conv_net/tblock_0/conv1/bias:0(2,)
tcn/temporal_conv_net/tblock_0/conv2/kernel:0(3, 3, 2, 2)
tcn/temporal_conv_net/tblock_0/conv2/bias:0(2,)
tcn/temporal_conv_net/tblock_0/conv3/kernel:0(3, 3, 1, 2)
tcn/temporal_conv_net/tblock_0/conv3/bias:0(2,)
tcn/temporal_conv_net/tblock_1/conv1/kernel:0(3, 3, 2, 4)
tcn/temporal_conv_net/tblock_1/conv1/bias:0(4,)
tcn/temporal_conv_net/tblock_1/conv2/kernel:0(3, 3, 4, 4)
tcn/temporal_conv_net/tblock_1/conv2/bias:0(4,)
tcn/temporal_conv_net/tblock_1/conv3/kernel:0(3, 3, 2, 4)
tcn/temporal_conv_net/tblock_1/conv3/bias:0(4,)
tcn/temporal_conv_net/tblock_2/conv1/kernel:0(3, 3, 4, 8)
tcn/temporal_conv_net/tblock_2/conv1/bias:0(8,)
tcn/temporal_conv_net/tblock_2/conv2/kernel:0(3, 3, 8, 8)
tcn/temporal_conv_net/tblock_2/conv2/b

In [38]:
def print_progress(step, loss, acc):
    print("Step {}, Loss={:.4f}, Accuracy={:.3f}".format(str(step), loss, acc))

In [39]:
def validation(epoch):    
    # Calculate batch loss and accuracy
    losses = []
    accuracies = []
    sess.run(iterator.initializer, feed_dict={sequences: val_data, labels: val_label_one})
    while True:
        try:
            # Run optimization
            loss, acc = sess.run([loss_op, accuracy], feed_dict={is_training: False})
            losses.append(loss)
            accuracies.append(acc)
        except tf.errors.OutOfRangeError:
            break
    loss_avg = sum(losses)/len(losses)
    acc_avg = sum(accuracies)/len(accuracies)
    print_progress("VALIDATION for epoch {}".format(epoch), loss_avg, acc_avg)
    return acc_avg

## Start training

In [40]:
from pathlib import Path
import random 
from datetime import datetime
path = "../../logs/tcn_sequence/"
log_dir = "{}{}".format(path, datetime.now().strftime("%Y%m%d_%H%M"))
Path(log_dir).mkdir(exist_ok=True, parents=True)
tb_writer = tf.summary.FileWriter(log_dir, graph)
config = tf.ConfigProto()
config.gpu_options.allow_growth = False
best_val_acc = 0.8
with tf.Session(graph=graph) as sess:
    # Run the initializer
    epoch, step = 0, 0
    sess.run([init, iterator.initializer], feed_dict={sequences: train_data, labels: train_label_one})
    while epoch < num_epochs:
        try: 
            sess.run(train_op, feed_dict={is_training: True})
            step = step +1 
            if step % int(batches_per_epoch/4) == 0 or step == 1:
                loss, acc = sess.run([loss_op, accuracy], feed_dict={is_training: True})
                print_progress(step, loss, acc)
                [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={is_training: True})
                tb_writer.add_summary(s, step)
        except tf.errors.OutOfRangeError:
            path
            epoch = epoch + 1
            val_acc = validation(epoch)           
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                save_path = saver.save(sess, "{}{}".format(path, "v1"))
                print("Model saved in path: %s" % save_path)
            sess.run(iterator.initializer, feed_dict={sequences: train_data, labels: train_label_one})
    print("Optimization Finished!")
    

Step 1, Loss=nan, Accuracy=0.000
Step 5785, Loss=nan, Accuracy=0.000


KeyboardInterrupt: 

# Validation with new sequences

In [61]:
import pandas as pd
data = pd.read_csv("..//data//test_sequences.csv", sep='\t', skipinitialspace=True)
data["Sequence"] = data.Sequence.str.ljust(500, '0')
letterToIndex = {'0': 0, 'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12,
                 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20}
data["Sequence_vector"] = [[letterToIndex[char] for char in val ] for index, val in data.Sequence.iteritems()]
test_data= np.asarray([ np.asarray(element) for element in data["Sequence_vector"].values])
test_data_for_tensorflow = np.append(test_data, np.zeros((batch_size-len(test_data), sequence_length)), axis=0).astype(int)
test_data_for_tensorflow.shape

(128, 500)

In [67]:
test_elements = np.array([1,2,3,4,5,6])
label_for_tensorflow = np.append(test_elements, np.zeros((batch_size-len(test_elements))), axis=0).astype(int)
label_for_tensorflow.shape

(128,)

In [68]:
s = tf.Session(graph=graph)
s.run(init)
saver.restore(s, "../logs/tcn_sequence/v1")

INFO:tensorflow:Restoring parameters from ../logs/tcn_sequence/v1


In [69]:
np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)

s.run(iterator.initializer, feed_dict={sequences: test_data_for_tensorflow, labels: label_for_tensorflow})
preds, ls = s.run([prediction, batch_labels], feed_dict={is_training: False})
count = 0
selected_ls = ls.take(np.argwhere(ls > 0))
selected_preds = preds.take(np.argwhere(ls > 0), axis=0)
for i in selected_ls.argsort(axis=0):
    if count == 0:
        print("\n\r")
        print("Oxidoreductases Transferases Hydrolases Lyases Isomerases Ligases")
    print(selected_preds[i])
    print(np.argmax(selected_preds[i]))
    count = count + 1

#     print( p["classes"]+1)



Oxidoreductases Transferases Hydrolases Lyases Isomerases Ligases
[[[0.00003311 0.00000058 0.         ... 0.00005921 0.00000001 0.00000002]]]
238
[[[0.         0.         0.         ... 0.         0.00000075 0.        ]]]
218
[[[0.0000794  0.         0.         ... 0.00029334 0.         0.        ]]]
548
[[[0.00000006 0.         0.         ... 0.00000005 0.00000172 0.        ]]]
1267
[[[0.00008943 0.         0.         ... 0.00000001 0.         0.        ]]]
471
[[[0. 0. 0. ... 0. 0. 0.]]]
245


In [31]:
selected_ls.argsort(axis=0)

array([[0],
       [4],
       [2],
       [1],
       [3]])

In [32]:
selected_ls

array([[1],
       [4],
       [3],
       [5],
       [2]], dtype=int32)

In [128]:
ls.sort()
ls

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5], dtype=int32)

In [130]:
np.argwhere(ls > 0)

array([[123],
       [124],
       [125],
       [126],
       [127]])